選擇這項題目的原因： 主要是我自己的程度還沒高到能自由使用Python 的各種工具與函數，作為初學者，我認為選擇競賽目標較單純的項目會比較得心應手，綜觀Kaggle上的所有競賽，去除上課做過的房價與鐵達尼號後，剩下的選項只下「太空鐵達尼號」與「生物環境對患有腎結石的可能性之影響」，考慮到想稍微突破自己的極限，故選擇了後者進行建模計算。

比賽簡介：該項比賽是基於已知的資料內，分析並計算環境中的各項指標（包含重力場、環境酸鹼值、尿液等數據）從雃建立模型判斷剩下的目標生物是否可能患有腎結石，其中target即為患有腎結石的可能性，將目標的資料集test對標已知的資料集train進行建模分析，計算出test中所有生物的target值。

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

先安裝需要的工具

In [ ]:
df = pd.read_csv("test.csv")
df2 = pd.read_csv("train.csv")

df是要測試的資料，df2是訓練的資料

In [ ]:
sns.countplot(x=df2['ph'],y=df2['target'])
sns.countplot(x=df2['gravity'],y=df2['target'])
sns.countplot(x=df2['osmo'],y=df2['target'])
sns.countplot(x=df2['cond'],y=df2['target'])
sns.countplot(x=df2['urea'],y=df2['target'])
sns.countplot(x=df2['calc'],y=df2['target'])

找出各項指標對於產生腎結石的可能性，利用圖形觀察各個指標影響的重要程度，各個指標的直為多少時有多少0或1

In [ ]:
df2.groupby('target').mean()

X=df2.iloc[:,1:7]  #X是所有可能的影響變因

X.head()
X.info()
Y = df2['target']    #Y是目標值

開始建模，先定義X,Y值，利用 df2.groupby('target').mean() 可得出結果
<img src="photo1.png" style="zoom:70%" />

也就是說，在gravity越高，ph值越低，osmo越大，cond越大，urea越大，calc越大的條件下，患有腎結石（target=1）的可能性越高

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, Y,test_size=0.3,random_state=54)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train) 
predictions=lr.predict(X_test)

predictions

先利用train.csv裡面的數據建立起「各項環境因素」轉換出「患有腎結石可能性」的模型，再將test.csv的檔案套入模型，所得之preditions即為test數據中的target值，為一個矩陣由多個0或1組成，共276個

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, recall_score, precision_score
accuracy_score(y_test,predictions)
recall_score(y_test,predictions)
precision_score(y_test,predictions)

計算經由建模後所算出的數據與真實數據的差距，如圖
<img src="photo3.png" style="zoom:70%" />

In [ ]:
import joblib
joblib.dump(lr,'sales_lr.pkl',compress=3 )

將建立的模型輸出

In [ ]:
import joblib 
model_pretrained=joblib.load('sales_lr.pkl')
import pandas as pd

df_test=pd.read_csv("test.csv")
df_test
df_test.drop(['id'],axis=1) 
df_test.head()

predictions2=model_pretrained.predict(df_test.drop(['id'],axis=1) )

predictions2

forSubmissionDF=pd.DataFrame(columns=['id','target'])
forSubmissionDF
forSubmissionDF['id']=range(414,690)
forSubmissionDF['target']=predictions2
forSubmissionDF

forSubmissionDF.to_csv('s.csv',index=False)

執行、輸出，將最後產生的s.csv上傳至kaggle競賽，獲得分數為77分
<img src="photo2.png" style="zoom:70%" />

對於這次的競賽參與，我花費最多時間的地方是在設定環境與弄清楚各種pytho的使用方法，很容易寫完後執行時一直遇到各種bug，時常一解決就要花上一個下午，好在最後都順利解決，當然以一個初學者的程度所建立的模型還不夠完整，我的模型所計算出的結果只能呈現0與1，也就是「是否可能患有腎結石」，然而實際上更完整的模型應該能夠準確值算出「患有腎結石的機率」，推論，應該有法能夠更精確的找出各項環境因素對於產生腎結石的重要程度，在建模的過程中我有試著觀察觀察平台上其他人所建立的計算模型，確實有人針對各項環境因素對target做圖找出影響程度，然而在我試圖理解該坐著的想法時卻發現他所使用的函式與思考方法遠超越我目前的能力，確實透過這項比賽我看見網路上各個高手的思維與建模方式，希望我未來也能寫出更準確的計算方式。